
# Analysis Comparison between Original and Expanded Lexicons on Cohort Analysis Task

Subreddit: bipolar

In [1]:
from pathlib import Path
import pandas as pd

from config import global_config
from lexicons2 import Values, ValuesExpanded, Liwc2015, Liwc2015Expanded
from spaces import WordEmbeddings, LabelEmbeddings

In [2]:
COHORT_NAME = 'bipolar'

In [3]:
cohort_corpus_path = global_config.reddit.submissions[COHORT_NAME]
cohort_models_path = f'{global_config.paths.models}/{COHORT_NAME}'

control_corpus_path = global_config.reddit.submissions['IAMA']
control_models_path = f'{global_config.paths.models}/IAMA'

In [4]:
cohort_ranks_csv = f'{global_config.paths.ranks}/{COHORT_NAME}_ranks.csv'
expanded_cohort_ranks_csv = f'{global_config.paths.ranks}/{COHORT_NAME}_ranks_expanded.csv'

control_ranks_csv = f'{global_config.paths.ranks}/IAMA_ranks.csv'
expanded_control_ranks_csv = f'{global_config.paths.ranks}/IAMA_ranks_expanded.csv'

relative_ranks_csv = f'{global_config.paths.ranks}/{COHORT_NAME}_relative-ranks.csv'
expanded_relative_ranks_csv = f'{global_config.paths.ranks}/{COHORT_NAME}_relative-ranks_expanded.csv'

Build Control Spaces

In [5]:
control_word_space = WordEmbeddings(corpus_path=control_corpus_path, model_path=control_models_path).build()

In [6]:
org_control_label_space = LabelEmbeddings(lexicons=[Liwc2015(), Values()], word_embeddings=control_word_space).build()

In [7]:
expanded_control_label_space = LabelEmbeddings(lexicons=[Liwc2015Expanded(), ValuesExpanded()], word_embeddings=control_word_space).build()

Build Cohort Spaces

In [8]:
cohort_word_space = WordEmbeddings(corpus_path=cohort_corpus_path, model_path=cohort_models_path).build()

In [9]:
org_cohort_label_space = LabelEmbeddings(lexicons=[Liwc2015(), Values()], word_embeddings=cohort_word_space).build()

In [10]:
expanded_cohort_label_space = LabelEmbeddings(lexicons=[Liwc2015Expanded(), ValuesExpanded()], word_embeddings=cohort_word_space).build()

Save and view results

In [11]:
def save_if_not_save_and_return(space: LabelEmbeddings, path: str):
	if not Path(path).exists():
		space.save_distances_to_csv(path)
	return pd.read_csv(path, names=['label_one', 'label_two', 'distance'])

control_ranks = save_if_not_save_and_return(org_control_label_space, control_ranks_csv)
expanded_control_ranks = save_if_not_save_and_return(expanded_control_label_space, expanded_control_ranks_csv)

cohort_ranks = save_if_not_save_and_return(org_cohort_label_space, cohort_ranks_csv)
expanded_cohort_ranks = save_if_not_save_and_return(expanded_cohort_label_space, expanded_cohort_ranks_csv)

In [12]:
org_cohort_label_space.compute_rank_deltas(org_control_label_space, relative_ranks_csv)
relative_ranks = pd.read_csv(relative_ranks_csv, names=['label_one', 'label_two', 'current_rank', 'control_rank', 'current_distance', 'control_distance', 'rank_delta', 'distance_delta'])

expanded_cohort_label_space.compute_rank_deltas(expanded_control_label_space, expanded_relative_ranks_csv)
expanded_relative_ranks = pd.read_csv(expanded_relative_ranks_csv, names=['label_one', 'label_two', 'current_rank', 'control_rank', 'current_distance', 'control_distance', 'rank_delta', 'distance_delta'])

View Stuff

In [13]:
relative_ranks

,label_one,label_two,current_rank,control_rank,current_distance,control_distance,rank_delta,distance_delta
0,liwc2015:function,values:learning,60,28,0.186554,0.178845,32,0.007709
1,liwc2015:function,values:feeling-good,68,43,0.297118,0.499940,25,-0.202823
2,liwc2015:function,values:truth,76,55,0.520907,0.713501,21,-0.192593
3,liwc2015:function,values:children,69,50,0.335780,0.680796,19,-0.345016
4,liwc2015:function,liwc2015:netspeak,80,62,0.660682,0.781215,18,-0.120532
...,...,...,...,...,...,...,...,...
6967,values:animals,values:religion,6,65,0.824966,0.965300,-59,-0.140334
6968,values:animals,liwc2015:filler,1,69,0.793116,0.980616,-68,-0.187500
6969,values:animals,liwc2015:male,12,82,0.841695,1.117406,-70,-0.275710
6970,values:animals,liwc2015:ingest,0,70,0.686929,0.981867,-70,-0.294938


In [14]:
expanded_relative_ranks

,label_one,label_two,current_rank,control_rank,current_distance,control_distance,rank_delta,distance_delta
0,liwc2015:function,liwc2015:work,57,39,0.071887,0.336770,18,-0.264883
1,liwc2015:function,liwc2015:pconcern,71,54,0.241987,0.624154,17,-0.382167
2,liwc2015:function,liwc2015:posemo,50,34,0.045427,0.259349,16,-0.213921
3,liwc2015:function,liwc2015:money,74,58,0.295761,0.681783,16,-0.386022
4,liwc2015:function,liwc2015:netspeak,83,68,0.660682,0.781215,15,-0.120532
...,...,...,...,...,...,...,...,...
7135,liwc2015:filler,values:social,10,54,0.438908,0.994057,-44,-0.555148
7136,liwc2015:filler,values:parents,0,51,0.413480,0.992307,-51,-0.578827
7137,liwc2015:filler,liwc2015:friend,7,61,0.438105,1.007286,-54,-0.569182
7138,liwc2015:filler,liwc2015:achiev,2,67,0.426735,1.031193,-65,-0.604458
